# 1. 데이터 집계

## groupby 메서드로 평균값 구하기

In [1]:
import pandas as pd

In [3]:
df=pd.read_csv('../data/gapminder.tsv', sep='\t')

In [4]:
avg_life_exp_by_year=df.groupby('year').lifeExp.mean()

In [5]:
print(avg_life_exp_by_year)

year
1952    49.057620
1957    51.507401
1962    53.609249
1967    55.678290
1972    57.647386
1977    59.570157
1982    61.533197
1987    63.212613
1992    64.160338
1997    65.014676
2002    65.694923
2007    67.007423
Name: lifeExp, dtype: float64


## 분할-반영-결합 과정 살펴보기

In [6]:
years=df.year.unique()

In [7]:
print(years)

[1952 1957 1962 1967 1972 1977 1982 1987 1992 1997 2002 2007]


In [8]:
y1952=df.loc[df.year==1952, :]

In [9]:
print(y1952.head())

        country continent  year  lifeExp       pop    gdpPercap
0   Afghanistan      Asia  1952   28.801   8425333   779.445314
12      Albania    Europe  1952   55.230   1282697  1601.056136
24      Algeria    Africa  1952   43.077   9279525  2449.008185
36       Angola    Africa  1952   30.015   4232095  3520.610273
48    Argentina  Americas  1952   62.485  17876956  5911.315053


In [10]:
y1952_mean=y1952.lifeExp.mean()

In [11]:
print(y1952_mean)

49.05761971830987


In [12]:
y1957=df.loc[df.year==1957,:]

In [13]:
y1957_mean=y1957.lifeExp.mean()

In [14]:
print(y1957_mean)

51.50740112676054


In [18]:
y1962=df.loc[df.year==1962,:]
y1962_mean=y1962.lifeExp.mean()

In [19]:
print(y1962_mean)

53.60924901408449


In [15]:
y2007=df.loc[df.year==2007,:]
y2007_mean=y2007.lifeExp.mean()

In [16]:
print(y2007_mean)

67.00742253521126


In [20]:
df2=pd.DataFrame({"year":[1952,1957,1962,2007],
                 "":[y1952_mean, y1957_mean, y1962_mean, y2007_mean]})

In [21]:
print(df2)

   year           
0  1952  49.057620
1  1957  51.507401
2  1962  53.609249
3  2007  67.007423


## 평균값을 구하는 사용자 함수와 groupby 메서드

In [25]:
def my_mean(values):
    n=len(values)
    sum=0
    for value in values:
        sum+=value
        
    return sum/n

In [27]:
agg_my_mean=df.groupby('year').lifeExp.agg(my_mean)

In [28]:
print(agg_my_mean)

year
1952    49.057620
1957    51.507401
1962    53.609249
1967    55.678290
1972    57.647386
1977    59.570157
1982    61.533197
1987    63.212613
1992    64.160338
1997    65.014676
2002    65.694923
2007    67.007423
Name: lifeExp, dtype: float64


## 두 개의 인잣값을 받아 처리하는 사용자 함수와 groupby 메서드

In [29]:
def my_mean_diff(values, diff_value):
    n=len(values)
    sum=0
    for value in values:
        sum+=value
    mean=sum/n
    return mean-diff_value

In [31]:
global_mean=df.lifeExp.mean()
print(global_mean)


59.47443936619713


In [32]:
agg_mean_diff=df.groupby('year').lifeExp.agg(my_mean_diff, diff_value=global_mean)

In [33]:
print(agg_mean_diff)

year
1952   -10.416820
1957    -7.967038
1962    -5.865190
1967    -3.796150
1972    -1.827053
1977     0.095718
1982     2.058758
1987     3.738173
1992     4.685899
1997     5.540237
2002     6.220483
2007     7.532983
Name: lifeExp, dtype: float64


## 집계 메서드를 리스트, 딕셔너리에 담아 전달하기

In [34]:
import numpy as np

In [36]:
gdf=df.groupby('year').lifeExp.agg([np.count_nonzero, np.mean, np.std])

In [37]:
print(gdf)

      count_nonzero       mean        std
year                                     
1952          142.0  49.057620  12.225956
1957          142.0  51.507401  12.231286
1962          142.0  53.609249  12.097245
1967          142.0  55.678290  11.718858
1972          142.0  57.647386  11.381953
1977          142.0  59.570157  11.227229
1982          142.0  61.533197  10.770618
1987          142.0  63.212613  10.556285
1992          142.0  64.160338  11.227380
1997          142.0  65.014676  11.559439
2002          142.0  65.694923  12.279823
2007          142.0  67.007423  12.073021


In [38]:
gdf_dict=df.groupby('year').agg({'lifeExp':'mean', 'pop':'median','gdpPercap':'median'})

In [39]:
print(gdf_dict)

        lifeExp         pop    gdpPercap
year                                    
1952  49.057620   3943953.0  1968.528344
1957  51.507401   4282942.0  2173.220291
1962  53.609249   4686039.5  2335.439533
1967  55.678290   5170175.5  2678.334741
1972  57.647386   5877996.5  3339.129407
1977  59.570157   6404036.5  3798.609244
1982  61.533197   7007320.0  4216.228428
1987  63.212613   7774861.5  4280.300366
1992  64.160338   8688686.5  4386.085502
1997  65.014676   9735063.5  4781.825478
2002  65.694923  10372918.5  5319.804524
2007  67.007423  10517531.0  6124.371109


# 2. 데이터 변환

## 표준 점수 계산하기

In [40]:
def my_zscore(x):
    return (x-x.mean())/x.std()

In [41]:
transform_z=df.groupby('year').lifeExp.transform(my_zscore)

In [42]:
print(transform_z.head())

0   -1.656854
1   -1.731249
2   -1.786543
3   -1.848157
4   -1.894173
Name: lifeExp, dtype: float64


In [43]:
print(df.shape)
print(transform_z.shape)

(1704, 6)
(1704,)


## 누락값을 평균값으로 처리하기

In [44]:
import seaborn as sns

In [45]:
import numpy as np

In [46]:
np.random.seed(42)

In [48]:
tips_10=sns.load_dataset('tips').sample(10)
tips_10.loc[np.random.permutation(tips_10.index)[:4], 'total_bill']=np.NaN

In [49]:
print(tips_10)

     total_bill   tip     sex smoker   day    time  size
58        11.24  1.76    Male    Yes   Sat  Dinner     2
73          NaN  5.00  Female    Yes   Sat  Dinner     2
138       16.00  2.00    Male    Yes  Thur   Lunch     2
193       15.48  2.02    Male    Yes  Thur   Lunch     2
235       10.07  1.25    Male     No   Sat  Dinner     2
117         NaN  1.50  Female     No  Thur   Lunch     2
154       19.77  2.00    Male     No   Sun  Dinner     4
55        19.49  3.51    Male     No   Sun  Dinner     2
87          NaN  4.00    Male     No  Thur   Lunch     2
9           NaN  3.23    Male     No   Sun  Dinner     2


In [50]:
count_sex=tips_10.groupby('sex').count()

In [51]:
print(count_sex)

        total_bill  tip  smoker  day  time  size
sex                                             
Male             6    8       8    8     8     8
Female           0    2       2    2     2     2


In [52]:
def fill_na_mean(x):
    avg=x.mean()
    return x.fillna(avg)

In [53]:
total_bill_group_mean=tips_10.groupby('sex').total_bill.transform(fill_na_mean)

In [54]:
tips_10['fill_total_bill']=total_bill_group_mean

In [55]:
print(tips_10)

     total_bill   tip     sex smoker   day    time  size  fill_total_bill
58        11.24  1.76    Male    Yes   Sat  Dinner     2        11.240000
73          NaN  5.00  Female    Yes   Sat  Dinner     2              NaN
138       16.00  2.00    Male    Yes  Thur   Lunch     2        16.000000
193       15.48  2.02    Male    Yes  Thur   Lunch     2        15.480000
235       10.07  1.25    Male     No   Sat  Dinner     2        10.070000
117         NaN  1.50  Female     No  Thur   Lunch     2              NaN
154       19.77  2.00    Male     No   Sun  Dinner     4        19.770000
55        19.49  3.51    Male     No   Sun  Dinner     2        19.490000
87          NaN  4.00    Male     No  Thur   Lunch     2        15.341667
9           NaN  3.23    Male     No   Sun  Dinner     2        15.341667


# 3. 데이터 필터링

## 데이터 필터링 사용하기 ─ filter 메서드

In [56]:
tips=sns.load_dataset('tips')

In [57]:
print(tips.shape)

(244, 7)


In [58]:
print(tips['size'].value_counts())

2    156
3     38
4     37
5      5
6      4
1      4
Name: size, dtype: int64


In [60]:
tips_filtered=tips.\
    groupby('size').\
    filter(lambda x: x['size'].count()>=30)

In [61]:
print(tips_filtered.shape)

(231, 7)


In [62]:
print(tips_filtered['size'].value_counts())

2    156
3     38
4     37
Name: size, dtype: int64


# 4. 그룹 오브젝트

## 그룹 오브젝트 저장하여 살펴보기

In [64]:
tips_10=sns.load_dataset('tips').sample(10,random_state=42)

In [65]:
print(tips_10)

     total_bill   tip     sex smoker   day    time  size
24        19.82  3.18    Male     No   Sat  Dinner     2
6          8.77  2.00    Male     No   Sun  Dinner     2
153       24.55  2.00    Male     No   Sun  Dinner     4
211       25.89  5.16    Male    Yes   Sat  Dinner     4
198       13.00  2.00  Female    Yes  Thur   Lunch     2
176       17.89  2.00    Male    Yes   Sun  Dinner     2
192       28.44  2.56    Male    Yes  Thur   Lunch     2
124       12.48  2.52  Female     No  Thur   Lunch     2
9         14.78  3.23    Male     No   Sun  Dinner     2
101       15.38  3.00  Female    Yes   Fri  Dinner     2


In [66]:
grouped=tips_10.groupby('sex')

In [67]:
print(grouped)

In [68]:
print(grouped.groups)

{'Male': Int64Index([24, 6, 153, 211, 176, 192, 9], dtype='int64'), 'Female': Int64Index([198, 124, 101], dtype='int64')}


## 그룹 오브젝트의 평균 구하기

In [69]:
avgs=grouped.mean()

In [70]:
print(avgs)

        total_bill       tip      size
sex                                   
Male         20.02  2.875714  2.571429
Female       13.62  2.506667  2.000000


In [71]:
print(tips_10.columns)

Index(['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size'], dtype='object')


## 그룹 오브젝트에서 데이터 추출하고 반복하기

In [72]:
female=grouped.get_group('Female')

In [73]:
print(female)

     total_bill   tip     sex smoker   day    time  size
198       13.00  2.00  Female    Yes  Thur   Lunch     2
124       12.48  2.52  Female     No  Thur   Lunch     2
101       15.38  3.00  Female    Yes   Fri  Dinner     2


In [75]:
for sex_group in grouped:
    print(sex_group)

('Male',      total_bill   tip   sex smoker   day    time  size
24        19.82  3.18  Male     No   Sat  Dinner     2
6          8.77  2.00  Male     No   Sun  Dinner     2
153       24.55  2.00  Male     No   Sun  Dinner     4
211       25.89  5.16  Male    Yes   Sat  Dinner     4
176       17.89  2.00  Male    Yes   Sun  Dinner     2
192       28.44  2.56  Male    Yes  Thur   Lunch     2
9         14.78  3.23  Male     No   Sun  Dinner     2)
('Female',      total_bill   tip     sex smoker   day    time  size
198       13.00  2.00  Female    Yes  Thur   Lunch     2
124       12.48  2.52  Female     No  Thur   Lunch     2
101       15.38  3.00  Female    Yes   Fri  Dinner     2)


In [79]:
for sex_group in grouped:
    print('the type is : {}\n'.format(type(sex_group)))
    print('the length is : {}\n'.format(len(sex_group)))
    
    first_element=sex_group[0]
    print('the first element is : {}\n'.format(first_element))
    print('it has a type of : {}\n'.format(type(sex_group[0])))
    
    second_element=sex_group[1]
    print('the second element is: \n{}\n'.format(second_element))
    print('it has a type of: {}\n'.format(type(second_element)))
    
    print('what we have:')
    print(sex_group)
    
    break

the type is : <class 'tuple'>

the length is : 2

the first element is : Male

it has a type of : <class 'str'>

the second element is: 
     total_bill   tip   sex smoker   day    time  size
24        19.82  3.18  Male     No   Sat  Dinner     2
6          8.77  2.00  Male     No   Sun  Dinner     2
153       24.55  2.00  Male     No   Sun  Dinner     4
211       25.89  5.16  Male    Yes   Sat  Dinner     4
176       17.89  2.00  Male    Yes   Sun  Dinner     2
192       28.44  2.56  Male    Yes  Thur   Lunch     2
9         14.78  3.23  Male     No   Sun  Dinner     2

it has a type of: <class 'pandas.core.frame.DataFrame'>

what we have:
('Male',      total_bill   tip   sex smoker   day    time  size
24        19.82  3.18  Male     No   Sat  Dinner     2
6          8.77  2.00  Male     No   Sun  Dinner     2
153       24.55  2.00  Male     No   Sun  Dinner     4
211       25.89  5.16  Male    Yes   Sat  Dinner     4
176       17.89  2.00  Male    Yes   Sun  Dinner     2
192       28

## 그룹 오브젝트 계산하고 살펴보기

In [80]:
bill_sex_time= tips_10.groupby(['sex','time'])
group_avg=bill_sex_time.mean()

In [81]:
print(group_avg)

               total_bill       tip      size
sex    time                                  
Male   Lunch    28.440000  2.560000  2.000000
       Dinner   18.616667  2.928333  2.666667
Female Lunch    12.740000  2.260000  2.000000
       Dinner   15.380000  3.000000  2.000000


In [82]:
print(type(group_avg))

<class 'pandas.core.frame.DataFrame'>


In [83]:
print(group_avg.columns)

Index(['total_bill', 'tip', 'size'], dtype='object')


In [84]:
print(group_avg.index)

MultiIndex([(  'Male',  'Lunch'),
            (  'Male', 'Dinner'),
            ('Female',  'Lunch'),
            ('Female', 'Dinner')],
           names=['sex', 'time'])


In [85]:
group_method=tips_10.groupby(['sex','time']).mean().reset_index()

In [86]:
print(group_method)

      sex    time  total_bill       tip      size
0    Male   Lunch   28.440000  2.560000  2.000000
1    Male  Dinner   18.616667  2.928333  2.666667
2  Female   Lunch   12.740000  2.260000  2.000000
3  Female  Dinner   15.380000  3.000000  2.000000


In [87]:
group_param=tips_10.groupby(['sex','time'], as_index=False).mean()

In [88]:
print(group_param)

      sex    time  total_bill       tip      size
0    Male   Lunch   28.440000  2.560000  2.000000
1    Male  Dinner   18.616667  2.928333  2.666667
2  Female   Lunch   12.740000  2.260000  2.000000
3  Female  Dinner   15.380000  3.000000  2.000000
